**420-A58-SF - Algorithmes d'apprentissage non supervisé - Hiver 2023 - Spécialisation technique en Intelligence Artificielle**<br/>
MIT License - Copyright (c) 2023 Mikaël Swawola
<br/>
![Travaux Pratiques - Algorithme Apriori](static/02-05-A1-banner.png)
<br/>
**Objectif: Cette séance de travaux pratiques consiste en l'implémentation de l'algorithme Apriori pour l'apprentissage des règles d'association sur le mini jeu de données PanierEpicerie**

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## 1 - Lecture du jeu de données

In [2]:
import numpy as np 
import pandas as pd
from helpers import print_itemsets, print_rules

**Exercice 1-1 - À l'aide de la librairie Pandas, lire le fichier de données `PanierEpicerie.csv`**

In [3]:
# Compléter cette cellule ~ 1-2 lignes de code
data = pd.read_csv('../../data/PanierEpicerie.csv')
data

,basket
0,"LAIT,PAIN,BISCUIT"
1,"PAIN,LAIT,BISCUIT,CORNFLAKES"
2,"PAIN,THE,CHOCOLAT"
3,"JAM,MAGGI,PAIN,LAIT"
4,"MAGGI,THE,BISCUIT"
5,"PAIN,THE,CHOCOLAT"
6,"MAGGI,THE,CORNFLAKES"
7,"MAGGI,PAIN,THE,BISCUIT"
8,"JAM,MAGGI,PAIN,THE"
9,"PAIN,LAIT"


**Exercice 1-2 - Combien d'items et de baskets sont contenus dans ce jeu de données ?**

In [4]:
# Votre réponse ici
# 20 baskets et 11 items

**Exercice 1-3 - Convertissez le jeu de données en liste de transactions (ou baskets). Chaque basket est une liste d'items (ou article)**

In [5]:
# Compléter cette cellule ~ 1-2 lignes de code

transactions = list(data["basket"].apply(lambda x:x.split(',')))
transactions

[['LAIT', 'PAIN', 'BISCUIT'],
 ['PAIN', 'LAIT', 'BISCUIT', 'CORNFLAKES'],
 ['PAIN', 'THE', 'CHOCOLAT'],
 ['JAM', 'MAGGI', 'PAIN', 'LAIT'],
 ['MAGGI', 'THE', 'BISCUIT'],
 ['PAIN', 'THE', 'CHOCOLAT'],
 ['MAGGI', 'THE', 'CORNFLAKES'],
 ['MAGGI', 'PAIN', 'THE', 'BISCUIT'],
 ['JAM', 'MAGGI', 'PAIN', 'THE'],
 ['PAIN', 'LAIT'],
 ['CAFE', 'JUS', 'BISCUIT', 'CORNFLAKES'],
 ['CAFE', 'JUS', 'BISCUIT', 'CORNFLAKES'],
 ['CAFE', 'SUCRE', 'CHOCOLAT'],
 ['PAIN', 'CAFE', 'JUS'],
 ['PAIN', 'SUCRE', 'BISCUIT'],
 ['CAFE', 'SUCRE', 'CORNFLAKES'],
 ['PAIN', 'SUCRE', 'CHOCOLAT'],
 ['PAIN', 'CAFE', 'SUCRE'],
 ['PAIN', 'CAFE', 'SUCRE'],
 ['THE', 'LAIT', 'CAFE', 'CORNFLAKES']]

In [6]:
flat_list = [item for transaction in transactions for item in transaction]
len(set(flat_list))

11

## 2 - Librarie Mlxtend

In [7]:
!pip install mlxtend

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


La librairie [Mlxtend](http://rasbt.github.io/mlxtend/) (machine learning extensions) propose une implémentation de l'algorithme Apriori. Nous allons donc mettre en oeuvre différentes fonctionnalités de cette librairie

**Exercice 2-1 - À l'aide de la classe [TransactionEncoder](http://rasbt.github.io/mlxtend/user_guide/preprocessing/TransactionEncoder/), encodez la liste des transactions au format requis par Mlxtend**

In [8]:
# Compléter cette cellule ~ 3-4 lignes de code
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_data = te.fit(transactions).transform(transactions)
te_data
df = pd.DataFrame(te_data,columns=te.columns_)
df

,BISCUIT,CAFE,CHOCOLAT,CORNFLAKES,JAM,JUS,LAIT,MAGGI,PAIN,SUCRE,THE
0,True,False,False,False,False,False,True,False,True,False,False
1,True,False,False,True,False,False,True,False,True,False,False
2,False,False,True,False,False,False,False,False,True,False,True
3,False,False,False,False,True,False,True,True,True,False,False
4,True,False,False,False,False,False,False,True,False,False,True
5,False,False,True,False,False,False,False,False,True,False,True
6,False,False,False,True,False,False,False,True,False,False,True
7,True,False,False,False,False,False,False,True,True,False,True
8,False,False,False,False,True,False,False,True,True,False,True
9,False,False,False,False,False,False,True,False,True,False,False


**Exercice 2-2 - Identifier les itemsets fréquents pour un support de 20%. Référez vous à la documentation de Mlxtend pour trouver la classe requise**

In [26]:
# Compléter cette cellule ~ 2-3 lignes de code
from mlxtend.frequent_patterns import apriori

freq = apriori(df, min_support=0.2, use_colnames=True).sort_values(by="support", ascending=False)
freq

,support,itemsets
6,0.65,(PAIN)
1,0.40,(CAFE)
0,0.35,(BISCUIT)
8,0.35,(THE)
3,0.30,(CORNFLAKES)
7,0.30,(SUCRE)
4,0.25,(LAIT)
5,0.25,(MAGGI)
2,0.20,(CHOCOLAT)
9,0.20,"(PAIN, BISCUIT)"


**Exercice 2-3 - De la même manière, identifier maintenant les règles d'association ayant un indice de confiance au moins de 0.3**

In [27]:
# Compléter cette cellule ~ 2-3 lignes de code
from mlxtend.frequent_patterns import association_rules

df_ar = association_rules(freq, metric = "confidence", min_threshold = 0.1).sort_values(by="confidence", ascending=False)
df_ar

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
6,(LAIT),(PAIN),0.25,0.65,0.2,0.800000,1.230769,0.0375,1.750000,0.250000
9,(MAGGI),(THE),0.25,0.35,0.2,0.800000,2.285714,0.1125,3.250000,0.750000
3,(CORNFLAKES),(CAFE),0.30,0.40,0.2,0.666667,1.666667,0.0800,1.800000,0.571429
4,(SUCRE),(CAFE),0.30,0.40,0.2,0.666667,1.666667,0.0800,1.800000,0.571429
10,(SUCRE),(PAIN),0.30,0.65,0.2,0.666667,1.025641,0.0050,1.050000,0.035714
1,(BISCUIT),(PAIN),0.35,0.65,0.2,0.571429,0.879121,-0.0275,0.816667,-0.174603
8,(THE),(MAGGI),0.35,0.25,0.2,0.571429,2.285714,0.1125,1.750000,0.865385
12,(THE),(PAIN),0.35,0.65,0.2,0.571429,0.879121,-0.0275,0.816667,-0.174603
2,(CAFE),(CORNFLAKES),0.40,0.30,0.2,0.500000,1.666667,0.0800,1.400000,0.666667
5,(CAFE),(SUCRE),0.40,0.30,0.2,0.500000,1.666667,0.0800,1.400000,0.666667


## 3 - Implémentation de l'algorithme Apriori

Le code ci-dessous représente une implémentation simple et de base (persque naïve) de l'algorihtme Apriori. 

**Exercice 3-1 - À l'aide des éléments vus en cours, compléter les différentes méthodes de la classe `Apriori` et retrouvez les résultats de l'exercice 2. Seule la méthode `generate_L` vous est donnée. Des fonctions helpers aidant à l'affichage `print_itemsets`, `print_rules` sont importées**

In [ ]:
class Apriori:
    
    def __init__(self, transactions, min_support, min_confidence):
        self.transactions = transactions # Baskets
        self.min_support = min_support # Le seuil de support
        self.min_confidence = min_confidence # La confiance minimale
        self.support_data = {}   
        
    def create_C1(self):
                
        # Completer le code ci-dessous ~ 1 ligne de code !
        
        # C1: set de frozenset
        
        #C1 = set([frozenset([item]) for transaction in transactions for item in transaction])
        
        C1 = set()
        for trans in self.transactions:
            for item in trans:
                C1.add(frozenset([item]))    
        
        return C1
    
    
    def generate_Lk_from_Ck(self, Ck):
        
        # Completer le code ci-dessous ~ 15-20 lignes de code
        
        Lk = set()
        items_count = {}
        
        for transaction in self.transactions:
            for items in Ck:
                if items.issubset(transaction):
                    if items not in items_count:
                        items_count[items] = 1
                    else:
                        items_count[items] += 1
        
        t_num = len(self.transactions)
        
        for items in items_count:
            support = items_count[items] / float(t_num)
            if support >= self.min_support:
                Lk.add(items)
                self.support_data[items] = support
        return Lk
    
        #Lk = set()
        
        # Trouver les fréquences des item(pair)
        #for r in Ck:                               # Chaque item(item set) a rechercher
        #    freq = 0
        #    for b in self.transactions:            # Chaque basket
        #        trouve = 0                         # Compteur pour identifier nb d'item trouvé
        #        for i in b:                        # Chaque item du basket
        #            if (i in r):
        #                trouve+=1
        #        if (trouve == len(r)):
        #            freq += 1
                    
        #    if (freq / len(self.transactions) >= self.min_support):
        #        Lk.add(r)
        #        self.support_data[r] = freq
                
        #return Lk
    
    
    
    
    def create_Ck(self, L1, Lksub1):
              
        # Completer le code ci-dessous ~ 6 lignes de code
        Ck = set()
        
        for item in L1:
            for itemset in Lksub1:
                union = item.union(itemset)
                if len(union) != len(itemset):
                    Ck.add(union)
                    
        #for itemset in Lksub1:
        #    for item in L1:
        #        union = itemset.union(item)
        #        if len(union) != len(itemset):
        #            Ck.add(union)
        
        
        
        
        return Ck
        
        
    def generate_L(self):
        """
        Génère tous les ensembles d'items fréquents
        Input:
            None
        Output:
            L: Liste des Lk.
        """
        
        self.support_data = {}
        
        C1 = self.create_C1()
        L1 = self.generate_Lk_from_Ck(C1)
        Lksub1 = L1.copy()
        L = []
        L.append(Lksub1)
        i = 2
        while True:
            Ci = self.create_Ck(L1, Lksub1)
            Li = self.generate_Lk_from_Ck(Ci)
            if Li:
                Lksub1 = Li.copy()
                L.append(Lksub1)
                i += 1
            else:
                break
        return L
        
        
    def generate_rules(self):
               
        L = self.generate_L()
        
        big_rule_list = []
        sub_set_list = []
        for i in range(0, len(L)):
            for freq_set in L[i]:
                for sub_set in sub_set_list:
                    if sub_set.issubset(freq_set):
                        # Compute the confidence
                        conf = self.support_data[freq_set] / self.support_data[freq_set - sub_set]
                        big_rule = (freq_set - sub_set, sub_set, conf)
                        if conf >= self.min_confidence and big_rule not in big_rule_list:
                            big_rule_list.append(big_rule)
                sub_set_list.append(freq_set)
        
        return big_rule_list

In [ ]:
model = Apriori(transactions, min_support=0.2, min_confidence=0.3)

In [ ]:
######################## POUR DEBUGGER ############################
C1 = model.create_C1()
C1
L1 = model.generate_Lk_from_Ck(C1)
L1
C2 = model.create_Ck(L1,L1)
L2 = model.generate_Lk_from_Ck(C2)
L2
C3 = model.create_Ck(L1,L2)
C3
L3 = model.generate_Lk_from_Ck(C3)
L3

In [ ]:
L = model.generate_L()

print_itemsets(L, model)

In [ ]:
rule_list = model.generate_rules()
print_rules(rule_list)

## Fin du TP